In [3]:
import pandas as pd

df = pd.read_csv('300_dishes.csv')
df.head()


,Dish Name,Ingredients,Cuisine,Type,Craving Type
0,Butter Chicken,"Chicken, Butter, Cream",North Indian,Curry,Rich
1,Paneer Tikka,"Paneer, Spices, Yogurt",North Indian,Appetizer,Spicy
2,Biryani,"Rice, Meat, Spices",Hyderabadi,Rice,Spicy
3,Samosa,"Potato, Flour, Spices",North Indian,Snack,Spicy
4,Masala Dosa,"Rice Batter, Potato",South Indian,South Indian,Spicy


In [4]:
# Combine columns into one description per dish
df["full_description"] = df.apply(lambda row: f"{row['Dish Name']}. It includes {row['Ingredients']}. It is a {row['Cuisine']} {row['Type']}, and is perfect when you're craving something {row['Craving Type']}.", axis=1)

# Preview
df["full_description"].head()


,full_description
0,"Butter Chicken. It includes Chicken, Butter, C..."
1,"Paneer Tikka. It includes Paneer, Spices, Yogu..."
2,"Biryani. It includes Rice, Meat, Spices. It is..."
3,"Samosa. It includes Potato, Flour, Spices. It ..."
4,"Masala Dosa. It includes Rice Batter, Potato. ..."


In [9]:
!pip install langchain openai faiss-cpu tiktoken

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
import os

In [10]:
# Set OpenAI API Key
# Replace "sk-proj-..." with your actual, valid OpenAI API key
os.environ["OPENAI_API_KEY"] = ""
# Create list of LangChain Documents
documents = [Document(page_content=desc) for desc in df["full_description"]]

# Create embeddings
embedding = OpenAIEmbeddings()

# Store in FAISS
vectorstore = FAISS.from_documents(documents, embedding)

# Save for later use
vectorstore.save_local("cravingbot_index")

In [12]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

# Load FAISS and create retriever
# Set allow_dangerous_deserialization=True to enable loading from the pickle file
vectorstore = FAISS.load_local("cravingbot_index", embedding, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# LLM + QA Chain
llm = OpenAI(temperature=0.7)
qa_chain = load_qa_chain(llm, chain_type="stuff")

# Query example
query = "I'm craving something spicy and creamy. What should I eat?"
docs = retriever.get_relevant_documents(query)
response = qa_chain.run(input_documents=docs, question=query)
print("🍽️ CravingBot Suggestion:", response)

<ipython-input-12-dc7b291a1e22>:10: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0.7)
<ipython-input-12-dc7b291a1e22>:11: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langc

🍽️ CravingBot Suggestion: 
You should try Chicken 65 or Chingri Malai Curry.
